In [1]:
# 예제1) Dataframe를 생성해봅시다.

#1. pandas를 import합니다.
import pandas as pd
#2. 3*2(row*column) 형태의 dataframe 을 만들겠습니다.
data = pd.DataFrame(data = [['RC',30],['XT',40],['ZT',50]], columns = ['grade','qty'])
#3. column명은 grade, qty로 명명해주고 index는 따로 추가하지 않습니다. 내용은 각자 채워주세요

In [2]:
# 예제2) 자동검사기 파일을 불러옵니다.

names = ['날짜','Lot','Frame수', '카메라 번호', '사용안함', 'INDEX', '불량유형','X','y', 
         'X-Size','Y-Size', '불량이미지','사용안함','사용안함','Value','흑점','조명밝기']

#0. 현재 작업 폴더를 확인하고 optical_python 내 files라는 폴더를 작업 폴더로 만들어주세요. 
import os
os.getcwd()
os.chdir('../dataset')

#1. files라는 폴더 내에 20160715DC02017.txt 파일을 read_csv 함수를 이용해서 불러와보세요. (변수 : data_1)
data_1 = pd.read_csv('20160715DC02017.txt', sep = '\t', names = names)

#2. files라는 폴더 내에 20160715DC02017.xlsx 파일을 read_excel 함수를 이용해서 불러와보세요. (변수 : data)
data = pd.read_excel('20160715DC02017.xlsx', header = None, names = names)

In [3]:
# 예제3) 위에서 불러온 data라는 변수의 기본적인 내용 확인 과정입니다.

#1. data의 상단 10줄을 head라는 함수를 사용해서 출력해보세요 
data.head()

#2. data의 기본 통계 수치를 describe 함수를 사용해서 출력해보세요 
data.describe()
#3. data의 전체 열, 3~행을 iloc을 사용해서 출력해보세요. 
data.iloc[:, 3:]

#4. data의 1열 1행의 값에 숫자 3을 대입해보세요. 값이 바뀌나요? 
data.iloc[0,0] = 3 

C:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [5]:
# 예제4) 행, 열 선택을 통한 원하는 데이터만 가져오기

#1. 위에서 불러온 data 변수에서 '불량유형' 행만 선택해서 나타내봅니다.
data['불량유형']
#2. '불량유형' 행의 값이 2305인 조건을 걸어보고, data 내 해당 조건을 만족하는 값들만 나타내보고 변수 data_2305에 저장합니다. 
data_2305 = data[data['불량유형'] == 2305]

In [6]:
names = ['날짜','Lot','Frame수', '카메라 번호', '사용안함', 'INDEX', '불량유형','X','y', 
         'X-Size','Y-Size', '불량이미지','사용안함','사용안함','Value','흑점','조명밝기']

# 예제5) 데이터 가공

#1. 위에서 불러온 검사기 data_2305에서 필요한 행만 남기고 지울 예정입니다. 
  # 행 이름(names)을 참고하셔서 '불량유형','X','y','X-Size','Y-Size','Value'를 제외한 행을 다 삭제해주세요 
data_2305 = data_2305.drop(['날짜','Lot','Frame수','카메라 번호','사용안함','INDEX','불량이미지','흑점','조명밝기'], axis = 1)    
    
#2. data_2305에 na값들을 넣어놨습니다. 아래 코드를 실행해서 값을 확인하고 다 지워보고(dropna) 원하시는 값으로 채워보세요 (fillna)

data_2305[data_2305.isnull().any(axis=1)] # null 값이 있는 열을 나타내줍니다. 

data_2305.fillna(value = 0, inplace = True)

#3. data_2305의 mean/std/min/max/corr을 확인해보세요 
data_2305.mean()
data_2305.std()
data_2305.min()
data_2305.max()
data_2305.corr()

#4. data_2305의 'Value' 행을 값의 범위 별로 나눈 후에 새로운 'cut'이라는 행에 저장하겠습니다. 
  # max로 최대값을 확인하신 후에 0부터 최대값까지 원하시는 만큼 쪼개보세요. 매칭할 label은 '0~30', '30~60'와 같은 범위로 나타내주세요.
data_2305['cut'] = pd.cut(data_2305['Value'], bins = [0, 40, 80, 120, 160, 200],
                          labels = ['0~40','40~80','80~120','120~160','160~200'])

In [10]:
# 예제6) 데이터 가공(apply, groupby)

#1. data_2305의 '불량유형' 행에 apply를 사용해서 숫자 60000보다 큰 값은 '약' 으로, 작은 값은 '강' 값을 반환하여 '강/약' 행에 저장합니다. 
data_2305['강/약'] = data_2305['불량유형'].apply(lambda x: '강' if x<60000 else '약')
#2. data_2305에 groupby 함수를 사용합니다. group을 묶을 행은 ['강/약, 'cut']이고, 적용 대상 행은 아무거나 고른 후에 count 함수를 써보세요..
data_2305.groupby(['강/약','cut'])['Value'].count()

강/약  cut    
강    0~40       175
     40~80      166
     80~120      38
     120~160     16
     160~200      7
Name: Value, dtype: int64

In [12]:
# 에제7) 데이터 합치기(merge)

#1. day_5_예제.xlsx 파일의 Sheet2 에서 A,B행을 불러옵니다.(변수 : prod_1)
prod_1 = pd.read_excel('day_5_예제.xlsx', parse_cols = ('A,B'), sheetname = 'Sheet2')

#2. day_5_예제.xlsx 파일의 Sheet2 에서 D,E행을 불러오고 6,7번 열을 불러오지 않습니다. (변수 : prod_2)
prod_2 = pd.read_excel('day_5_예제.xlsx', parse_cols = ('D,E'), skiprows = (6,7), sheetname = 'Sheet2')

#3. prod_1, prod_2를 '연신' 행에 대해서 merge합니다. (inner, outer, left, right) 
pd.merge(prod_1, prod_2, how = 'outer', on = '연신')

#4. 3의 결과를 보고 어떤 식으로 작동하는 지 확인합니다. 

,원재료,연신,코팅
0,20160101CM01001,20170517CE02001,20170517CC03001
1,20160101CM01001,20170517CE02001,20170517CC03002
2,20160101CM01002,20170517CE02001,20170517CC03001
3,20160101CM01002,20170517CE02001,20170517CC03002
4,20160101CM01003,20170517CE02002,20170517CC03003
5,20160101CM01003,20170517CE02003,20170517CC03004
6,20160101CM01003,20170517CE02003,20170517CC03005
7,20160101CM01004,20170517CE02003,20170517CC03004
8,20160101CM01004,20170517CE02003,20170517CC03005
9,20160101CM01005,20170517CE02004,NaN


In [20]:
# 예제8) 데이터 합치기(concat)

#1. data_5_예제.xlsx 파일의 Sheet3에서 A,B / D, E/ G, H/ J, K 행을 각각 불러옵니다. 
   # 변수 : coat_1, coat_2, coat_3, coat_4
coat_1 = pd.read_excel('day_5_예제.xlsx', parse_cols = ('A,B'), sheetname = 'Sheet3')
coat_2 = pd.read_excel('day_5_예제.xlsx', parse_cols = ('D,E'), sheetname = 'Sheet3')
coat_3 = pd.read_excel('day_5_예제.xlsx', parse_cols = ('G,H'), sheetname = 'Sheet3')
coat_4 = pd.read_excel('day_5_예제.xlsx', parse_cols = ('J,K'), sheetname = 'Sheet3')

#2. coat_1 과 coat_2를 concat을 사용해서 합쳐보세요. 어떤 방향으로 합치는게 보기 좋나요? 
pd.concat([coat_1, coat_2], axis = 0)

#3. for 문을 이용해서 coat_1부터 4까지 합쳐줍니다. 
   # 데이터를 다른 변수(바구니)에 담아야 loop를 돌릴 수 있습니다. 
   # 그리고 concat을 해서 어디에 저장해야 될 지 생각해보세요

coat_total = pd.DataFrame()
total = [coat_1, coat_2, coat_3, coat_4]
for coat in total:
    coat_total = pd.concat([coat_total, coat], axis = 0)

print(coat_total)

,코팅,생산량
0,20170517CC03001,20
1,20170517CC03002,30
2,20170517CC03003,40
3,20170517CC03004,10
4,20170517CC03005,2
0,20170516CC03001,30
1,20170516CC03002,21
2,20170516CC03003,56
3,20170516CC03004,28
4,20170516CC03005,98


In [22]:
# 예제9) 유용한 데이터 가공(melt) 

#1. data_5_예제.xlsx 파일의 Sheet4 데이터를 가져옵니다. 
ext_1 = pd.read_excel('day_5_예제.xlsx', sheetname = 'Sheet4')

#2. melt를 사용해서 외부컬, 내부컬 행을 하나의 행으로 합쳐줍니다. 
pd.melt(ext_1, id_vars = ['연신'], value_vars = ['외부컬','내부컬'], var_name = ['컬종류'], value_name = '컬수치')

,연신,컬종류,컬수치
0,20170517CE02001,외부컬,1
1,20170517CE02002,외부컬,2
2,20170517CE02003,외부컬,3
3,20170517CE02004,외부컬,4
4,20170517CE02005,외부컬,5
5,20170517CE02006,외부컬,6
6,20170517CE02007,외부컬,7
7,20170517CE02008,외부컬,8
8,20170517CE02001,내부컬,3
9,20170517CE02002,내부컬,4
